In [1]:
import os, sys, codecs
import numpy as np
import cv2
from skimage.io import imread

from sklearn.cross_validation import KFold
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
# Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

# model build
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils

from keras.optimizers import Adam

from keras.models import model_from_json
'''
0 cat : 6
1 chipmunk : 7
2 Dog : 4
3 fox : 3
4 giraffe : 8
5 guinea pigs 0
6 hyena : 10
7 reindeer : 9
8 sika deer : 2
9 squirrel : 1
10 weasel : 11
11 wolf : 5
'''

class Model_average():
    def __init__(self):
        self.model=[]
        self.proba=None

    def add_model(self, jsonPath, weightPath):
        # model have to be loaded weight
        json_file = open(jsonPath, 'r')
        loaded_model_json = json_file.read()
        json_file.close()

        # struc model
        model = model_from_json(loaded_model_json)
        # loading model weight
        model.load_weights(weightPath)
        # add model
        self.model.append(model)

    def predict_proba(self, iuPut):
        for model in self.model:
            predictProba = model.predict_proba(iuPut)[0]
            if self.proba is None:
                self.proba=predictProba
            else:
                self.proba+=predictProba
        result = self.proba/float(len(self.model))
        self.proba=None
        return [result]

def auto_resized(img,size):
    '''size = (width,height)'''
    size = tuple(size)
    resize_img = cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)
    return resize_img

def TrainFilePath(folderPath, constrain=None, **kargs):
    '''
    (1) Output filepath and calssName
    (2) folderPath
          --label_1
           -- xxx.jpg
    '''
    assert folderPath[-1]!='/'
    if constrain is None:
        constrain = ('avi', 'mp4','png','jpg','jpeg')
    for (rootDir, dirNames, fileNames) in os.walk(folderPath):
        for fileName in fileNames:
            if fileName.split('.')[-1] in constrain:
                yield (os.path.join(rootDir, fileName))


#img_channels = 3
def genTrX(filePath, resolution, img_channels=3):
    assert type(resolution) == tuple
    img = auto_resized(imread(filePath),resolution)  #conf['sliding_size']
    if img_channels==1:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif img_channels==3:
        try:
            img = img[:,:,:3]
        except:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    return img

# Create mapping list
mapping_dict = {}
a = [6,7,4,3,8,0,10,9,2,1,11,5]
for i in range(12):
    mapping_dict[i]=a[i]

Using Theano backend.
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/kentc/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.12-64/tmpvyxjrk/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/kentc/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-2.7.12-64/tmpvyxjrk/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970M (CNMeM is disabled, cuDNN 5103)
C:\Users\kentc\Anaconda2\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [10]:
testing_file = 'D:\\2016bot_cv\\fox'

In [13]:
img_rows= 92
img_cols= 92
img_channels=3
samplewise_center=None
samplewise_std_normalization=None

# Setting model path and load
model = Model_average()
model.add_model(jsonPath = '..\\hub\\model\\2016_bot_005.json',
                weightPath= '..\\hub\\model\\2016_bot_0050.h5')
model.add_model(jsonPath = '..\\hub\\model\\2016_bot_006.json',
                weightPath= '..\\hub\\model\\2016_bot_0062.h5')


In [14]:

iii=0
for imgPath in TrainFilePath(testing_file):
    if iii>100:
        break
    image = genTrX(imgPath, (img_rows,img_cols), img_channels)
    image = np.asarray(image, dtype = np.uint8)

    image = image.astype('float32')

    # Since the training process contrain the augmentation
    if samplewise_center:
        image -= np.mean(image, axis=0, keepdims=True) # for theano
    if samplewise_std_normalization:
        image /= (np.std(image, axis=0, keepdims=True) + 1e-7) # for theano

    image = image.reshape(1, img_channels, img_rows, img_cols)

    predictProba = model.predict_proba(image)[0]


    top2_ALL = predictProba.argsort()[-2:][::-1]
    top1 = top2_ALL[0]
    top1_p = round(predictProba[top1],6)
    top2 = top2_ALL[1]
    top2_p = round(predictProba[top2],6)
    top1_l = mapping_dict[top1]
    top2_l = mapping_dict[top2]
    imgName = imgPath.split('\\')[-1].split('.')[0]
    print ('{}\t{}\t{}\t{}\t{}\n'.format(imgName,
                                     top1_l,
                                     top1_p,
                                     top2_l,
                                     top2_p))
    iii+=1
print ('finished ')

1/1 [==============================] - 0s
00077fad1949480ea94cdb09c90a1e0c	2	0.157156	10	0.156191

1/1 [==============================] - 0s
000825d16ffc48a5a7d9cb488e0cbeb3	2	0.157156	10	0.156191

1/1 [==============================] - 0s
00104abdee084da3bf09ebfa22ab4e44	2	0.157156	10	0.156191

1/1 [==============================] - 0s
001437efea954dab84ddc88eb5c68a8c	2	0.157156	10	0.156191

1/1 [==============================] - 0s
0018e7dc116640b694cb8b4fe5673b2d	2	0.157156	10	0.156191

1/1 [==============================] - 0s
001e5a885cbb4e21a9641517b6ba5ef9	2	0.157156	10	0.156191

1/1 [==============================] - 0s
0029e38b94dd447780b2571fa2133feb	10	0.358791	7	0.331436

1/1 [==============================] - 0s
002bdef4cdfe4c39abe43c835a366704	2	0.157156	10	0.156191

1/1 [==============================] - 0s
002e6bfc6109450a8f6ccebde38ef6a0	2	0.157156	10	0.156191

1/1 [==============================] - 0s
003bbf0f1a0a454db7b48e2b0b125557	2	0.157156	10	0.156191

1/1 [=====